In [ ]:

#| default_exp cb2



In [ ]:
#| export
import os
import json

def load_or_compute_embeddings(compute_embeddings):
    """
    Checks for required precomputed files and either loads them or computes and saves them.
    
    Parameters:
      compute_embeddings (callable): A function that computes and returns the embeddings.
    
    Returns:
      dict: The loaded or newly computed embeddings.
    """
    required_files = [
        "graph_chunk_entity_relation.graphml",
        "kv_store_text_chunks.json",
        "kv_store_doc_status.json",
        "vdb_chunks.json",
        "kv_store_full_docs.json",
        "vdb_entities.json",
        "kv_store_llm_response_cache.json",
        "vdb_relationships.json"
    ]
    
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print("Missing files detected:", missing_files)
        # Compute embeddings using the provided function
        embeddings = compute_embeddings()
        # Save results to one or more of the required files
        # (You may need to adapt this to your file structure)
        with open("vdb_chunks.json", "w", encoding="utf-8") as f:
            json.dump(embeddings, f)
        # Save other files as needed...
    else:
        print("All precomputed files found. Loading cached embeddings...")
        with open("vdb_chunks.json", "r", encoding="utf-8") as f:
            embeddings = json.load(f)
    
    return embeddings


In [1]:
#| export
#| label: rag-query-system
#| fig-cap: RAG Query Interface
#| description: Cached query system that prevents redundant executions

import asyncio
import nest_asyncio
import os
import inspect
import logging
import csv
from pathlib import Path

from lightrag import LightRAG, QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.utils import EmbeddingFunc
from lightrag.kg.shared_storage import initialize_pipeline_status

nest_asyncio.apply()

WORKING_DIR = "./dickens"
SENTINEL_FILE = Path(".rag_initialized")

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

def _ensure_workspace():
    """Create workspace directory if missing"""
    os.makedirs(WORKING_DIR, exist_ok=True)

async def initialize_rag():
    """Initialize RAG system with caching check"""
    if SENTINEL_FILE.exists():
        logging.info("RAG system already initialized - using cached instance")
        return LightRAG.from_cache(WORKING_DIR)
    
    logging.info("Initializing fresh RAG instance")
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=ollama_model_complete,
        llm_model_name="deepseek-r1:1.5b1a",
        llm_model_max_async=4,
        llm_model_max_token_size=32768,
        llm_model_kwargs={
            "host": "http://localhost:11434",
            "options": {"num_ctx": 32768},
        },
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=8192,
            func=lambda texts: ollama_embed(
                texts, embed_model="nomic-embed-text", host="http://localhost:11434"
            ),
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()
    SENTINEL_FILE.touch()
    return rag

async def print_stream(stream):
    """Handle streaming responses"""
    async for chunk in stream:
        print(chunk, end="", flush=True)

def _load_data():
    """Load CSV data with existence check"""
    csv_path = Path("./42a.csv")
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV file not found at {csv_path}")
    
    with csv_path.open("r", encoding="utf-8") as f:
        return "\n".join(
            f"Topic: {row['Topic']}\nKey Concepts/Themes: {row['Key Concepts/Themes']}"
            for row in csv.DictReader(f)
        )

def execute_queries(rag):
    """Execute all predefined queries"""
    queries = {
        "Naive Search": QueryParam(mode="naive"),
        "Local Search": QueryParam(mode="local"),
        "Global Search": QueryParam(mode="global"),
        "Hybrid Search": QueryParam(mode="hybrid")
    }
    
    for name, param in queries.items():
        print(f"\n{name}:")
        result = rag.query(
            "Can you summarize how each topic combines concepts of self-expression, beauty, and resource management?",
            param=param
        )
        print(result if not inspect.isasyncgen(result) else asyncio.run(print_stream(result)))

def main():
    """Main execution flow with cache check"""
    _ensure_workspace()
    
    if SENTINEL_FILE.exists() and not os.getenv("FORCE_RERUN"):
        logging.info("Skipping execution - use FORCE_RERUN=1 to override")
        return
    
    rag = asyncio.run(initialize_rag())
    rag.insert(_load_data())
    execute_queries(rag)

if __name__ == "__main__":
    main()

ImportError: cannot import name 'LightRAG' from 'lightrag' (/opt/homebrew/Caskroom/miniforge/base/envs/lightrag/lib/python3.10/site-packages/lightrag/__init__.py)

In [ ]:
import nbdev; nbdev.nbdev_export()